# Test files for Recurrent Neural Network

In [ ]:
import numpy as np
import tensorflow as tf
import keras
from keras import layers
import matplotlib.pyplot as plt

### Loading data

In [ ]:
from load_data import load_and_transform_data

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = load_and_transform_data(SEED=42)

In [ ]:
# Check shape of subsets
print(f"X train shape:\t", X_train.shape)
print(f"y train shape:\t", y_train.shape)
print(f"X val shape:\t", X_val.shape)
print(f"y val shape:\t", y_val.shape)
print(f"X test shape:\t", X_test.shape)
print(f"y test shape:\t", y_test.shape)

### Initial testing for RNN, LSTM and GRU

In [ ]:
RNN = keras.Sequential()                          # Create a base sequential model
RNN.add(keras.Input(shape=(480, 480)))            # Set the input shape
RNN.add(layers.SimpleRNN(units=240))      # Add a simple RNN layer
RNN.add(layers.Dense(units=2, activation="softmax")) # Output layer

# Get a summary of model configuration
RNN.summary()

In [ ]:
LSTM = keras.Sequential()                          # Create a base sequential model
LSTM.add(keras.Input(shape=(480, 480)))            # Set the input shape
LSTM.add(layers.LSTM(units=240))      # Add a LSTM layer
LSTM.add(layers.Dense(units=2, activation="softmax")) # Output layer

# Get a summary of model configuration
LSTM.summary()

In [ ]:
GRU = keras.Sequential()                          # Create a base sequential model
GRU.add(keras.Input(shape=(480, 480)))            # Set the input shape
GRU.add(layers.GRU(units=240))      # Add a GRU layer
GRU.add(layers.Dense(units=2, activation="softmax")) # Output layer

# Get a summary of model configuration
GRU.summary()

### Model Training

In [ ]:
batch_size = 32
epochs = 10

In [ ]:
RNN.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

RNN_trainer = RNN.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=epochs)

RNN_test_loss, RNN_test_acc = RNN.evaluate(X_test, y_test)
print(f'RNN Test Accuracy: {RNN_test_acc}')

In [ ]:
LSTM.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

LSTM_trainer = LSTM.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=epochs)

LSTM_test_loss, LSTM_test_acc = LSTM.evaluate(X_test, y_test)
print(f'LSTM Test Accuracy: {LSTM_test_acc}')

In [ ]:
GRU.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

GRU_trainer = GRU.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=epochs)

GRU_test_loss, GRU_test_acc = GRU.evaluate(X_test, y_test)
print(f'GRU Test Accuracy: {GRU_test_acc}')

### Plotting Results

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Accuracy plot
ax1.plot(RNN_trainer.history['val_accuracy'], label='RNN Val Accuracy')
ax1.plot(LSTM_trainer.history['val_accuracy'], label='LSTM Val Accuracy')
ax1.plot(GRU_trainer.history['val_accuracy'], label='GRU Val Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.set_title('Model Accuracy')
ax1.legend()
ax1.grid(True)

# Loss plot
ax2.plot(RNN_trainer.history['val_loss'], label='RNN Val Loss')
ax2.plot(LSTM_trainer.history['val_loss'], label='LSTM Val Loss')
ax2.plot(GRU_trainer.history['val_loss'], label='GRU Val Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.set_title('Model Loss')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
y_pred = RNN.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_test, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, 
                               display_labels=['NORMAL', 'PNEUMONIA'])
disp.plot(cmap='Blues')
plt.title('Confusion Matrix - Test Set')
plt.show()

# 3. Print test metrics
test_loss, test_acc = RNN.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')
print(f'Test Loss: {test_loss:.4f}')

In [ ]:
y_pred = LSTM.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_test, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, 
                               display_labels=['NORMAL', 'PNEUMONIA'])
disp.plot(cmap='Blues')
plt.title('Confusion Matrix - Test Set')
plt.show()

# 3. Print test metrics
test_loss, test_acc = LSTM.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')
print(f'Test Loss: {test_loss:.4f}')

In [ ]:
y_pred = GRU.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_test, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, 
                               display_labels=['NORMAL', 'PNEUMONIA'])
disp.plot(cmap='Blues')
plt.title('Confusion Matrix - Test Set')
plt.show()

# 3. Print test metrics
test_loss, test_acc = GRU.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')
print(f'Test Loss: {test_loss:.4f}')

## Initial Rough Grid Search

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import itertools
from tqdm import tqdm
import pandas as pd
import os
from load_data import load_and_transform_data

# Speed optimizations
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'

# Create results directory
os.makedirs("CSV results", exist_ok=True)

# Hyperparameters
units = [15, 30, 60, 120, 240]
dropout = [0.0, 0.15, 0.3]
epochs = [10]
n_layers = [1, 2, 3]
layer_types = [layers.SimpleRNN, layers.LSTM, layers.GRU]

# Generate all layer configurations
def generate_layer_configs(n_layers, layer_types, units):
    """Generate all possible layer type combinations"""
    if n_layers == 1:
        return [(typ,) for typ in layer_types]
    elif n_layers == 2:
        return list(itertools.product(layer_types, layer_types))
    elif n_layers == 3:
        return list(itertools.product(layer_types, layer_types, layer_types))
    return []

In [ ]:
# Load data
from load_data import load_and_transform_data
X_train, y_train, X_val, y_val, X_test, y_test = load_and_transform_data(SEED=42)

In [ ]:
# Calculate total combinations
layer_configs = []
for n_lay in n_layers:
    layer_configs.extend([(n_lay, config) for config in generate_layer_configs(n_lay, layer_types, units)])

total_combinations = len(units) * len(dropout) * len(epochs) * len(layer_configs)
print(f"Total combinations: {total_combinations}")

# Store results
results1 = []

# Train all models
for idx, (u, drop, ep, (n_lay, layer_config)) in enumerate(tqdm(
    itertools.product(units, dropout, epochs, layer_configs), 
    total=total_combinations)):
    
    # Clear previous sessions
    tf.keras.backend.clear_session()
    
    # Build model based on layer configuration
    model = keras.Sequential()
    model.add(keras.Input(shape=(480, 480)))
    
    # Add layers
    for i, layer_type in enumerate(layer_config):
        return_sequences = (i < len(layer_config) - 1)  # True for all but last layer
        model.add(layer_type(units=u, dropout=drop, return_sequences=return_sequences))
    
    # Output layer
    model.add(layers.Dense(units=2, activation="softmax"))
    
    # Compile
    model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

    # Train with early stopping
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, verbose=0)
    
    trainer = model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                       epochs=ep, verbose=0, callbacks=[early_stop])

    # Get metrics
    val_acc = trainer.history['val_accuracy'][-1]
    val_loss = trainer.history['val_loss'][-1]
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

    # Create layer type string
    layer_names = [typ.__name__ for typ in layer_config]
    layer_str = ' -> '.join(layer_names)

    # Store results
    results1.append({
        'n-Layers': n_lay,
        'Layer-Config': layer_str,
        'n-Units': u,
        'Dropout': drop,
        'n-Epochs': ep,
        'Val Accuracy': val_acc,
        'Val Loss': val_loss,
        'Test Accuracy': test_acc,
        'Test Loss': test_loss
    })
    
    # Save progress
    pd.DataFrame(results1).to_csv("CSV results/GRU_results_rough_temp.csv", index=False)

# Save final results
df_results1 = pd.DataFrame(results1)
df_results1.to_csv("CSV results/GRU_results_rough.csv", index=False)

In [ ]:
# Show top configurations
top_20 = df_results1.nlargest(20, 'Val Accuracy')
print(f"\nComplete! {len(results1)} models trained")
print("\nTop 20 Configurations:")
print(top_20[['Layer-Config', 'n-Units', 'Dropout', 'Val Accuracy', 'Test Accuracy']])

In [ ]:
# Analyze results
df_results1 = pd.read_csv("CSV results/GRU_results_rough.csv")

# Top 50
top_16 = df_results1.nlargest(14, 'Val Accuracy')
print("\nTop 50 configurations:")
print(top_16[['N-type', 'n-Units', 'Dropout', 'n-Epochs', 'Val Accuracy', 'Test Accuracy']])

# Best model
print("\nBest model:")
print(df_results1.iloc[df_results1['Val Accuracy'].idxmax()])

In [ ]:
# Value counts
print("\nTop 50 distributions:")
for col in ['N-type', 'n-Units', 'Dropout', 'n-Epochs', 'Unroll']:
    print(f"\n{col}:")
    print(top_16[col].value_counts())